In [33]:
import sqlite3
import pandas as pd


connexion = sqlite3.connect("../DB/vivino.db")
cursor = connexion.cursor()

In [34]:
%load_ext sql

%config SqlMagic.displaylimit = None

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


displaylimit: Value None will be treated as 0 (no limit)

In [35]:
# Connecting to the database
%sql sqlite:///../DB/vivino.db

In [36]:
%%sql
-- Lists all the tables in a particular database
SELECT name FROM sqlite_master WHERE type='table';

Running query in 'sqlite:///../DB/vivino.db'

name
countries
grapes
keywords
regions
most_used_grapes_per_country
toplists
wines
vintages
keywords_wine
vintage_toplists_rankings


In [5]:
# %%sql
# DROP TABLE IF EXISTS flavor_groups

In [6]:
# # %%sql
# DROP TABLE IF EXISTS wineries

In [37]:
%%sql
SELECT name FROM sqlite_master WHERE type='table'

Running query in 'sqlite:///../DB/vivino.db'

name
countries
grapes
keywords
regions
most_used_grapes_per_country
toplists
wines
vintages
keywords_wine
vintage_toplists_rankings


### 1. We want to highlight 10 wines to increase our sales. Which ones should we choose and why?

In [38]:
%%sql

SELECT v.name AS wine_name,
       MIN(v.year) AS year,
       MIN(v.price_euros) AS price_euros,
       MAX(v.ratings_average) AS ratings_average,
       MAX(wines.ratings_count) AS ratings_count
FROM vintages v
JOIN wines ON v.wine_id = wines.id
JOIN vintage_toplists_rankings ON v.id = vintage_toplists_rankings.vintage_id
WHERE v.price_euros < 300
GROUP BY wines.name
ORDER BY MIN(vintage_toplists_rankings.rank) ASC
LIMIT 10;

Running query in 'sqlite:///../DB/vivino.db'

wine_name,year,price_euros,ratings_average,ratings_count
San Marzano 60 Sessantanni Old Vines Primitivo di Manduria 2018,2017,24.75,4.5,94289
Tommasi Amarone della Valpolicella Classico 2018,2017,49.8,4.4,77515
Buena Vista Chateau Buena Vista Cabernet Sauvignon Napa Valley 2018,2018,61.95,4.4,13415
Guerrieri Guerriero della Terra 2019,2019,27.4,4.4,10185
Charles Melton Nine Popes 2015,2015,114.6,4.4,2118
Bodegas Tradición Palo Cortado Tradición Vors 30 Years Sherry,N.V.,99.95,4.4,1050
Arzuaga Reserva Ribera del Duero 2018,2018,39.55,4.4,11233
Pago de Carraovejas Tinto 2020,2019,42.95,4.5,65625
Château des Tours Vacqueyras 2011,2011,250.0,4.4,3412
Dal Forno Romano Valpolicella Superiore Monte Lodoletta 2004,2004,243.75,4.5,13024


In [7]:
query1 = """SELECT v.name AS wine_name,
       MIN(v.year) AS year,
       MIN(v.price_euros) AS price_euros,
       MAX(v.ratings_average) AS ratings_average,
       MAX(wines.ratings_count) AS ratings_count
FROM vintages v
JOIN wines ON v.wine_id = wines.id
JOIN vintage_toplists_rankings ON v.id = vintage_toplists_rankings.vintage_id
WHERE v.price_euros < 300
GROUP BY wines.name
ORDER BY MIN(vintage_toplists_rankings.rank) ASC
LIMIT 10;
"""
df = pd.read_sql_query (query1, connexion)
df.to_csv("../CSV/csv_10_wines.csv", index=False)
df.head()


,wine_name,year,price_euros,ratings_average,ratings_count
0,San Marzano 60 Sessantanni Old Vines Primitivo...,2017,24.75,4.5,94289
1,Tommasi Amarone della Valpolicella Classico 2018,2017,49.80,4.4,77515
2,Buena Vista Chateau Buena Vista Cabernet Sauvi...,2018,61.95,4.4,13415
3,Guerrieri Guerriero della Terra 2019,2019,27.40,4.4,10185
4,Charles Melton Nine Popes 2015,2015,114.60,4.4,2118


### 2. Blmabqm,mqsd

### 3. We would like to give awards to the best wineries. Come up with 3 relevant ones. Which wineries should we choose and why?

In [56]:
%%sql

SELECT 
    t.vintage_id, 
    vintages.name, 
    wines.url, 
    t.rank, 
    t.previous_rank, 
    (t.previous_rank - t.rank) AS rank_difference
FROM vintage_toplists_rankings t
JOIN vintages
    ON t.vintage_id = vintages.id
JOIN wines
    ON vintages.wine_id = wines.id
WHERE t.rank = 1
GROUP BY vintages.name, t.vintage_id, wines.url, t.rank, t.previous_rank
ORDER BY rank_difference DESC
LIMIT 3;

Running query in 'sqlite:///../DB/vivino.db'

vintage_id,name,url,rank,previous_rank,rank_difference
156154317,Arzuaga Reserva Ribera del Duero 2018,https://joyvino.be/products/arzuaga-reserva-2018,1,14,13
160549459,Guerrieri Guerriero della Terra 2019,http://www.vivino.com/wines/165224957,1,12,11
162964134,Pago de Carraovejas Tinto 2020,https://www.grandcruwijnen.nl/2020-pago-de-carraovejas,1,11,10


In [58]:
query3 = """SELECT 
    t.vintage_id, 
    vintages.name, 
    wines.url, 
    t.rank, 
    t.previous_rank, 
    (t.previous_rank - t.rank) AS rank_difference
FROM vintage_toplists_rankings t
JOIN vintages
    ON t.vintage_id = vintages.id
JOIN wines
    ON vintages.wine_id = wines.id
WHERE t.rank = 1
GROUP BY vintages.name, t.vintage_id, wines.url, t.rank, t.previous_rank
ORDER BY rank_difference DESC
LIMIT 3;
"""
df = pd.read_sql_query (query3, connexion)
df.to_csv("../CSV/csv_top3_wineries_top1.csv", index=False)
df.head()

,vintage_id,name,url,rank,previous_rank,rank_difference
0,156154317,Arzuaga Reserva Ribera del Duero 2018,https://joyvino.be/products/arzuaga-reserva-2018,1,14,13
1,160549459,Guerrieri Guerriero della Terra 2019,http://www.vivino.com/wines/165224957,1,12,11
2,162964134,Pago de Carraovejas Tinto 2020,https://www.grandcruwijnen.nl/2020-pago-de-car...,1,11,10


In [59]:
query3_1 = """SELECT 
    t.vintage_id, 
    vintages.name, 
    wines.url, 
    t.rank, 
    t.previous_rank, 
    (t.previous_rank - t.rank) AS rank_difference
FROM vintage_toplists_rankings t
JOIN vintages
    ON t.vintage_id = vintages.id
JOIN wines
    ON vintages.wine_id = wines.id
GROUP BY vintages.name, t.vintage_id, wines.url, t.rank, t.previous_rank
ORDER BY rank_difference DESC
LIMIT 3;
"""
df = pd.read_sql_query (query3_1, connexion)
df.to_csv("../CSV/csv_top3_wineries_bestgap.csv", index=False)
df.head()

,vintage_id,name,url,rank,previous_rank,rank_difference
0,156154317,Arzuaga Reserva Ribera del Duero 2018,https://joyvino.be/products/arzuaga-reserva-2018,1,14,13
1,156105768,Château Pape Clément Pessac-Léognan (Grand Cru...,https://www.grandcruwijnen.nl/2018-chateau-pap...,2,15,13
2,156132702,Tommasi Amarone della Valpolicella Classico 2018,https://www.seulementproduitsitaliens.be/vino/...,4,17,13


### 4. We detected that a big cluster of customers likes a specific combination of tastes. We identified a few keywords that match these tastes: coffee, toast, green apple, cream, and citrus (note that these keywords are case sensitive ⚠️). We would like you to find all the wines that are related to these keywords. Check that at least 10 users confirm those keywords, to ensure the accuracy of the selection. Additionally, identify an appropriate group name for this cluster.

In [ ]:
# OK

### 5. We would like to select wines that are easy to find all over the world. Find the top 3 most common grapes all over the world and for each grape, give us the the 5 best rated wines.

In [105]:
%%sql

SELECT
    v.name,
    v.ratings_average,
    regions.name AS region,
    regions.country_code,
    countries.name AS country,
    most_used_grapes_per_country.grape_id AS grapes,
    grapes.name AS grape_name
FROM vintages v
JOIN wines ON v.wine_id = wines.id
JOIN regions ON wines.region_id = regions.id
JOIN countries ON regions.country_code = countries.code
JOIN most_used_grapes_per_country ON countries.code = most_used_grapes_per_country.country_code
JOIN grapes ON most_used_grapes_per_country.grape_id = grapes.id
ORDER BY v.ratings_average DESC
LIMIT 10;


Running query in 'sqlite:///../DB/vivino.db'

name,ratings_average,region,country_code,country,grapes,grape_name
Château Doisy-Daëne L'Extravagant de Doisy-Daëne Sauternes 2001,4.9,Sauternes,fr,France,5,Chardonnay
Château Doisy-Daëne L'Extravagant de Doisy-Daëne Sauternes 2001,4.9,Sauternes,fr,France,10,Merlot
Château Doisy-Daëne L'Extravagant de Doisy-Daëne Sauternes 2001,4.9,Sauternes,fr,France,14,Pinot Noir
Biondi-Santi Brunello di Montalcino Riserva 1955,4.9,Brunello di Montalcino,it,Italie,5,Chardonnay
Biondi-Santi Brunello di Montalcino Riserva 1955,4.9,Brunello di Montalcino,it,Italie,10,Merlot
Biondi-Santi Brunello di Montalcino Riserva 1955,4.9,Brunello di Montalcino,it,Italie,16,Sangiovese
Château Lafite Rothschild Pauillac (Premier Grand Cru Classé) 2003,4.8,Pauillac,fr,France,5,Chardonnay
Château Lafite Rothschild Pauillac (Premier Grand Cru Classé) 2003,4.8,Pauillac,fr,France,10,Merlot
Château Lafite Rothschild Pauillac (Premier Grand Cru Classé) 2003,4.8,Pauillac,fr,France,14,Pinot Noir
Krug Clos du Mesnil 1998,4.8,Champagne,fr,France,5,Chardonnay


In [109]:
%%sql
SELECT
    v.name AS vintage_name,
    v.ratings_average,
    regions.name AS region,
    regions.country_code,
    countries.name AS country,
    STRING_AGG(grapes.name, ', ') AS grape_names -- Concatenating grape names
FROM vintages v
JOIN wines ON v.wine_id = wines.id
JOIN regions ON wines.region_id = regions.id
JOIN countries ON regions.country_code = countries.code
JOIN most_used_grapes_per_country ON countries.code = most_used_grapes_per_country.country_code
JOIN grapes ON most_used_grapes_per_country.grape_id = grapes.id
GROUP BY v.name, v.ratings_average, regions.name, regions.country_code, countries.name
ORDER BY v.ratings_average DESC
LIMIT 10;


Running query in 'sqlite:///../DB/vivino.db'

vintage_name,ratings_average,region,country_code,country,grape_names
Biondi-Santi Brunello di Montalcino Riserva 1955,4.9,Brunello di Montalcino,it,Italie,"Chardonnay, Merlot, Sangiovese"
Château Doisy-Daëne L'Extravagant de Doisy-Daëne Sauternes 2001,4.9,Sauternes,fr,France,"Chardonnay, Merlot, Pinot Noir"
Artadi Viña El Pison 2018,4.8,Vino de España,es,Espagne,"Cabernet Sauvignon, Tempranillo, Garnacha"
Casanova di Neri Cerretalto Brunello di Montalcino 2016,4.8,Brunello di Montalcino,it,Italie,"Chardonnay, Merlot, Sangiovese"
Château Ausone Saint-Émilion Grand Cru (Premier Grand Cru Classé) 2018,4.8,Saint-Émilion Grand Cru,fr,France,"Chardonnay, Merlot, Pinot Noir"
Château Lafite Rothschild Pauillac (Premier Grand Cru Classé) 2003,4.8,Pauillac,fr,France,"Chardonnay, Merlot, Pinot Noir"
Château Latour Grand Vin Pauillac (Premier Grand Cru Classé) 2003,4.8,Pauillac,fr,France,"Chardonnay, Merlot, Pinot Noir"
Château d'Yquem Sauternes 1967,4.8,Sauternes,fr,France,"Chardonnay, Merlot, Pinot Noir"
Château d'Yquem Sauternes 2015,4.8,Sauternes,fr,France,"Chardonnay, Merlot, Pinot Noir"
Château d'Yquem Sauternes 2017,4.8,Sauternes,fr,France,"Chardonnay, Merlot, Pinot Noir"


In [111]:
%%sql
WITH top_grapes AS (
    SELECT
        grape_id,
        COUNT(DISTINCT country_code) AS country_count
    FROM most_used_grapes_per_country
    GROUP BY grape_id
    ORDER BY country_count DESC
    LIMIT 3
),
ranked_wines AS (
    SELECT
        wines.id AS wine_id,
        wines.name AS wine_name,
        v.name AS vintage_name,
        v.ratings_average,
        regions.name AS region_name,
        countries.name AS country_name,
        grapes.name AS grape_name,
        ROW_NUMBER() OVER (PARTITION BY grapes.id ORDER BY v.ratings_average DESC) AS wine_rank
    FROM wines
    JOIN vintages v ON wines.id = v.wine_id
    JOIN regions ON wines.region_id = regions.id
    JOIN countries ON regions.country_code = countries.code
    JOIN most_used_grapes_per_country mugpc ON countries.code = mugpc.country_code
    JOIN grapes ON mugpc.grape_id = grapes.id
    WHERE grapes.id IN (SELECT grape_id FROM top_grapes)
)
SELECT
    wine_name,
    vintage_name,
    ratings_average,
    region_name,
    country_name,
    grape_name
FROM ranked_wines
WHERE wine_rank <= 5
ORDER BY grape_name, ratings_average DESC
LIMIT 15;

Running query in 'sqlite:///../DB/vivino.db'

wine_name,vintage_name,ratings_average,region_name,country_name,grape_name
Harlan Estate Red,Harlan Estate Harlan Estate Red 1998,4.8,Napa Valley,États-Unis,Cabernet Sauvignon
Cabernet Sauvignon,Scarecrow Cabernet Sauvignon 2015,4.8,Rutherford,États-Unis,Cabernet Sauvignon
IX Estate Red,Colgin IX Estate Red 2016,4.8,Napa Valley,États-Unis,Cabernet Sauvignon
Viña El Pison,Artadi Viña El Pison 2018,4.8,Vino de España,Espagne,Cabernet Sauvignon
Cabernet Sauvignon,Leonetti Cabernet Sauvignon 2006,4.7,Walla Walla Valley,États-Unis,Cabernet Sauvignon
L'Extravagant de Doisy-Daëne Sauternes,Château Doisy-Daëne L'Extravagant de Doisy-Daëne Sauternes 2001,4.9,Sauternes,France,Chardonnay
Brunello di Montalcino Riserva,Biondi-Santi Brunello di Montalcino Riserva 1955,4.9,Brunello di Montalcino,Italie,Chardonnay
Pauillac (Premier Grand Cru Classé),Château Lafite Rothschild Pauillac (Premier Grand Cru Classé) 2003,4.8,Pauillac,France,Chardonnay
Clos du Mesnil,Krug Clos du Mesnil 1998,4.8,Champagne,France,Chardonnay
Le Mesnil Blanc de Blancs (Cuvée S) Brut Champagne,Salon Le Mesnil Blanc de Blancs (Cuvée S) Brut Champagne 1996,4.8,Champagne Grand Cru 'Le Mesnil-sur-Oger',France,Chardonnay


### 6. We would like to create a country leaderboard. Come up with a visual that shows the average wine rating for each country. Do the same for the vintages.

### 7. One of our VIP clients likes Cabernet Sauvignon and would like our top 5 recommendations. Which wines would you recommend to him?